In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re
import re
import sys
import os
import dotenv 
dotenv.load_dotenv()


key = os.getenv("token")

In [2]:
url = "https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchHotels"
querystring = {
    "geoId": "187449",                                          #fechas parametros de la API
    "checkIn": "2024-11-01",
    "checkOut": "2024-11-03",                                   #Dos adultos
    "pageNumber": "1",
    "adults": "2",
    "currencyCode": "EUR"
}
headers = {
    "x-rapidapi-key": key,
    "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
if response.status_code == 200:
    res = response.json()
    
    #como res es un diccionario que incluye dos elementos datas , entonces tenemos que entrar en el segundo data que es una
    #lista de diccionarios

    if 'data' in res and 'data' in res['data']: 
        hoteles = res['data']['data']

type(hoteles)


list

In [3]:
data = []
for hotel in hoteles:
        title = hotel.get('title')
            
        badge = hotel.get('badge', {})
        badge_size = badge.get('size')
        bubble_rating = hotel.get('bubbleRating', {})
        bubble_rating_value = bubble_rating.get('rating')
        price = hotel.get('priceForDisplay')
        provider = hotel.get('provider')
        card_photos = hotel.get('cardPhotos', [])
        commerce_info = hotel.get('commerceInfo', {})
        external_url = commerce_info.get('externalUrl')
        details = hotel.get('details', {})
        details_text = details.get('text')

        data.append({
                'Title': title,
                'Badge Size': badge_size,
                'Bubble Rating Value': bubble_rating_value,
                'Details Text': details_text,
                'Price': price,
                'Provider': provider,
                'External URL': external_url
                
            })
        
df = pd.DataFrame(data)


df.head(2)
  

,Title,Badge Size,Bubble Rating Value,Details Text,Price,Provider,External URL
0,Pension Alvaro,None,3.5,None,€65,Booking.com,https://www.tripadvisor.in/Commerce?p=BookingC...
1,1. Barceló Oviedo Cervantes,SMALL,4.5,None,€168,Booking.com,https://www.tripadvisor.in/Commerce?p=BookingC...


In [4]:
type(res)   

dict

In [5]:
df.isnull().sum()

Title                   0
Badge Size             24
Bubble Rating Value     0
Details Text           37
Price                   1
Provider                0
External URL            1
dtype: int64

In [6]:
df_Asturias = df.fillna('Nan')

In [7]:
df_Asturias.columns = ['Nombre Hotel','Talla','Puntuacion media','Detalles extra','Precio','Proveedor','URL']

In [8]:
df_Asturias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Nombre Hotel      37 non-null     object 
 1   Talla             37 non-null     object 
 2   Puntuacion media  37 non-null     float64
 3   Detalles extra    37 non-null     object 
 4   Precio            37 non-null     object 
 5   Proveedor         37 non-null     object 
 6   URL               37 non-null     object 
dtypes: float64(1), object(6)
memory usage: 2.2+ KB


In [3]:
df_hoteles = pd.read_csv('../Datos/tripadvaisor_Asturias.csv')
df_hoteles.head(2)

,Unnamed: 0,Nombre Hotel,Talla,Puntuacion media,Detalles extra,Precio,Proveedor,URL
0,0,HOTEL VALLE LAS LUIÑAS,Nan,4.5,Nan,88.0,Booking.com,https://www.tripadvisor.in/Commerce?p=BookingC...
1,1,1. Barceló Oviedo Cervantes,SMALL,4.5,Nan,151.0,Booking.com,https://www.tripadvisor.in/Commerce?p=BookingC...


In [5]:
df_hoteles['Ciudad'] = 'Asturias'

In [6]:
df_hoteles.dtypes

Unnamed: 0            int64
Nombre Hotel         object
Talla                object
Puntuacion media    float64
Detalles extra       object
Precio              float64
Proveedor            object
URL                  object
Ciudad               object
dtype: object

In [7]:
df_hoteles.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df_hoteles['Precio'] = df_hoteles['Precio'].str.replace(',', '.')
df_hoteles['Precio'] = df_hoteles['Precio'].str.replace('€', '')

In [9]:
df_hoteles['Precio'] = df_hoteles['Precio'].astype(float)

In [10]:
df_hoteles.head(2)
df_hoteles.dtypes

Nombre Hotel         object
Talla                object
Puntuacion media    float64
Detalles extra       object
Precio              float64
Proveedor            object
URL                  object
Ciudad               object
dtype: object

In [11]:
df_hoteles.to_csv('../Datos/tripadvaisor_Asturias.csv')